In [ ]:
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
from chronos import ChronosPipeline
import yfinance as yf
from typing import Any, Hashable
import pandas as pd
import requests
import re
from datetime import datetime
from nselib import capital_market
import numpy as np

In [ ]:
def get_ticker_from_company_name(company_name: str) -> str | None:
    """
    Searches for a stock ticker, prioritizing Indian exchanges (NSE/BSE) to ensure INR pricing.

    Args:
        company_name: The full or partial name of the company to search for.

    Returns:
        The best matching ticker symbol (str) or None if no symbol is found.
    """
    try:
        search_results = yf.Search(company_name)
        if not search_results.quotes:
            return None

        # 1. Try to find an NSE (.NS) or BSE (.BO) match first
        for quote in search_results.quotes:
            symbol = quote.get("symbol", "")
            if symbol.endswith(".NS") or symbol.endswith(".BO"):
                return symbol

        # 2. If no Indian match is found, return the best global match (likely USD/EUR)
        return search_results.quotes[0].get("symbol")

    except Exception as e:
        print(f"An error occurred during search: {e}")
        return None

In [ ]:
def clean_column_names(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = (df.columns.str.strip().str.replace(r'\s+', ' ', regex=True))
    return df

In [ ]:
def _clean_stock_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.reset_index()
    df.columns = [c.lower().replace(" ", "_") for c in df.columns]
    if "date" in df.columns:
        df = df.rename(columns={"date": "datetime"})
    df["datetime"] = pd.to_datetime(df["datetime"]).dt.tz_localize(None)
    drop_cols = [c for c in ["dividends", "stock_splits"] if c in df.columns]
    df = df.drop(columns=drop_cols)
    return df


In [ ]:
def get_bse_nse_data():
    nse_url = "https://archives.nseindia.com/content/equities/EQUITY_L.csv"
    nse_df = pd.read_csv(nse_url)
    nse_df = clean_column_names(nse_df)
    nse_df_final = nse_df[["NAME OF COMPANY", "SYMBOL"]]
    # nse_df_final.head()
    bse_url = (
    "https://api.bseindia.com/BseIndiaAPI/api/ListofScripData/w"
    "?Group=&Scripcode=&industry=&segment=Equity&status=Active"
    )

    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json",
        "Referer": "https://www.bseindia.com/",
        "Origin": "https://www.bseindia.com"
    }

    response = requests.get(bse_url, headers=headers, timeout=30)
    response.raise_for_status()
    data = response.json()
    bse_df = pd.DataFrame(data)
    bse_df = clean_column_names(bse_df)
    bse_df_final = bse_df.loc[bse_df["Status"].str.lower() == "active", ["Issuer_Name", "scrip_id"]].reset_index(drop=True)
    bse_df_final = bse_df_final.rename(columns={"Issuer_Name":"NAME OF COMPANY","scrip_id":"SYMBOL"})
    # bse_df_final.head()
    bse_nse_df = pd.concat([nse_df_final, bse_df_final]).drop_duplicates().reset_index(drop=True)
    bse_nse_df = bse_nse_df.drop_duplicates(subset=['SYMBOL'])
    bse_nse_df = bse_nse_df.sort_values(by='NAME OF COMPANY').reset_index(drop=True)
    # merged_df.tail(10)
    return bse_nse_df

In [ ]:
def company_list():
    list_of_company = get_bse_nse_data()["NAME OF COMPANY"].to_list()
    return list_of_company

In [ ]:
def get_ticker(company_name):
    df = get_bse_nse_data()
    company_to_symbol = dict(zip(df['NAME OF COMPANY'], df['SYMBOL']))
    symbol = company_to_symbol.get(company_name)
    return symbol

In [ ]:
def get_stock_data_using_yfinance(company_name: str, interval: str = "1d"):
    """
    Fetch stock data (Daily or Hourly) and return as a dataframe.

    Parameters
    ----------
    company_name : str
        The full or partial name of the company (e.g., 'Apple' or 'Microsoft'). 
    interval : str, optional
        '1d' (default) or '1h'

    Returns
    -------
    DataFrame
    """

    # 1d	max	    Full available history
    # 1h	730d	Last ~2 years only

    if interval not in ["1d", "1h"]:
        print(f"Invalid interval: {interval}")
        return []

    period = "730d" if interval == "1h" else "max"
    
    # 2. Define the strategy: List of methods to get the ticker
    # We store functions in a list to execute them sequentially
    ticker_strategies = [(get_ticker, "Local Lookup"), (get_ticker_from_company_name, "API Search")]
    
    df_result = pd.DataFrame()
    found_ticker = None

    # 3. Iterate through strategies
    for strategy_func, strategy_name in ticker_strategies:
        try:
            # Get candidate ticker
            candidate_ticker = strategy_func(company_name)
            
            if not candidate_ticker:
                continue # Try next strategy
            
            # Attempt to fetch data
            ticker_obj = yf.Ticker(candidate_ticker)
            temp_df = ticker_obj.history(period=period, interval=interval, auto_adjust=False)
            
            # Check if data is valid
            if not temp_df.empty:
                df_result = temp_df
                found_ticker = candidate_ticker
                # print(f"Success: Found data for '{company_name}' using {strategy_name} ({candidate_ticker})")
                break # Stop searching, we found data
            
        except Exception as e:
            print(f"Error trying {strategy_name} for '{company_name}': {e}")
            continue

    # 4. Final Data Processing
    if df_result.empty:
        return f"No data found for '{company_name}' after trying all methods."#, "Error"

    try:
        df_result = df_result.reset_index()        
        df_result.columns = [c.lower().replace(" ", "_") for c in df_result.columns]
        
        # Standardize Date/Datetime column
        if 'date' in df_result.columns:
            df_result = df_result.rename(columns={'date': 'datetime'})
            
        # Remove Timezone info for compatibility
        if 'datetime' in df_result.columns:
            df_result['datetime'] = pd.to_datetime(df_result['datetime']).dt.tz_localize(None)
            
        # Drop unnecessary columns if they exist
        cols_to_drop = ["dividends", "stock_splits", "capital_gains"]
        df_result = df_result.drop(columns=[c for c in cols_to_drop if c in df_result.columns], axis=1)
        
        return df_result#, "Success"

    except Exception as e:
        return f"Error processing data for '{company_name}': {e}"#, "Error"

In [ ]:
def get_stock_data_using_nselib(company_name):
    try:
        columns={"Date":"datetime", "OpenPrice":"open", "HighPrice":"high", "LowPrice":"low", "ClosePrice":"close", "AveragePrice":"adj_close", "TotalTradedQuantity":"volume"}
        # to_day = datetime.today().strftime("%d-%m-%Y")
        stock_data = capital_market.price_volume_data(symbol=get_ticker(company_name), from_date='01-01-2000', to_date=datetime.today().strftime("%d-%m-%Y"))
        stock_data = stock_data[list(columns.keys())]
        stock_data = stock_data.rename(columns=columns)
        stock_data["datetime"] = pd.to_datetime(stock_data["datetime"], format="%d-%b-%Y").dt.strftime("%Y-%m-%d")
        return stock_data
    except Exception as er:
        return f"No data found for '{company_name}'. \nError occured {er}"

In [ ]:
def get_whole_stock_data(company_name, interval: str = "1d"):
    stock_data = get_stock_data_using_yfinance(company_name, interval)
    if interval == "1d" and not isinstance(stock_data, pd.DataFrame):
        stock_data = get_stock_data_using_nselib(company_name)
        print("Stock data using nselib")
    print("Stock data using Yfinance")
    stock_data["open"] = stock_data["open"].apply(lambda x: float(x.replace(",", "")) if type(x) is str else x)
    stock_data["high"] = stock_data["high"].apply(lambda x: float(x.replace(",", ""))if type(x) is str else x)
    stock_data["low"] = stock_data["low"].apply(lambda x: float(x.replace(",", ""))if type(x) is str else x)
    stock_data["close"] = stock_data["close"].apply(lambda x: float(x.replace(",", ""))if type(x) is str else x)
    stock_data["adj_close"] = stock_data["adj_close"].apply(lambda x: float(x.replace(",", ""))if type(x) is str else x)
    stock_data["volume"] = stock_data["volume"].apply(lambda x: int(x.replace(",", ""))if type(x) is str else x)
    return stock_data

In [ ]:
# company_list()[2000]
data = get_whole_stock_data('J.Kumar Infraprojects Limited') #'J.Kumar Infraprojects Limited'
data

In [ ]:
company_name = company_list()
get_ticker(company_name[2000]), company_name[2000]

In [ ]:
"JKIL", 'J.Kumar Infraprojects Limited'

In [ ]:
import plotly.graph_objects as go

def plot_predictions_with_history(original_df, prediction_df, interval, lookback=20):
    """
    Creates an interactive plot using Plotly.
    1. Hover over points to see Price/Date.
    2. Zoom in/out to see specific dates on the X-axis.
    """
    
    history_subset = original_df.tail(lookback).copy()
    
    # Get the connection point (last historical data point)
    last_hist_point = history_subset.iloc[-1]
    
    # Create the prediction line data, starting from the last history point
    pred_dates = [last_hist_point['datetime']] + list(prediction_df['datetime'])
    pred_prices = [last_hist_point['adj_close']] + list(prediction_df['predicted_price'])
    
    # 2. Create the Figure
    fig = go.Figure()

    # --- Trace 1: History (Blue Line) ---
    fig.add_trace(go.Scatter(
        x=history_subset['datetime'],
        y=history_subset['adj_close'],
        mode='lines+markers', # Shows both line and dots
        name='History',
        line=dict(color="#1f77b4", width=2), # Matplotlib Blue
        marker=dict(size=6)
    ))

    # --- Trace 2: Forecast (Red Dashed Line) ---
    fig.add_trace(go.Scatter(
        x=pred_dates,
        y=pred_prices,
        mode='lines+markers',
        name='Forecast',
        line=dict(color='#d62728', width=2, dash='dash'), # Matplotlib Red
        marker=dict(symbol='x', size=8)
    ))

    # 3. Clean Layout (The "Matplotlib" Look)
    fig.update_layout(
        title=f'Prediction: {interval.capitalize()} Interval',
        xaxis_title='Date / Time',
        yaxis_title='Price',
        template="plotly_white", # Clean white background like Matplotlib
        hovermode="x unified",   # Shows a nice box with values when you hover
        showlegend=True
    )

    # 4. X-Axis Formatting (Crucial Fixes)
    # - rangeslider_visible=False: REMOVES the bottom mini-graph
    # - tickformat: Ensures hours are shown for hourly data
    if interval == 'hourly':
        fig.update_xaxes(
            rangeslider_visible=False, 
            tickformat="%d-%b %H:%M",  # e.g., 10-Jan 14:00
            nticks=20 # Try to show more labels by default
        )
    else:
        fig.update_xaxes(
            rangeslider_visible=False,
            tickformat="%Y-%m-%d",     # e.g., 2025-01-10
            nticks=20
        )

    fig.show()

In [ ]:
from ARIMA_GARCH import run_statistical_prediction
from fb_prophet_model import run_prophet_prediction
from DL_Model import run_DL_prediction
from ML_Model import run_ML_prediction
from Transformer_model import run_transformer_prediction

In [ ]:
df_daily = get_whole_stock_data('J.Kumar Infraprojects Limited', interval="1d")
predicted_daily_prices = run_transformer_prediction(df_daily, 'daily', 5)

print("Plotting Daily Data...")
plot_predictions_with_history(
    original_df=df_daily, 
    prediction_df=predicted_daily_prices, 
    interval='daily', 
    lookback=20
)

In [ ]:
df_hourly = get_stock_data(company_name[1000], interval="1h")
predicted_hourly_prices = run_dl_prediction(df_hourly, 'hourly', 5, "GRU")

print("Plotting Hourly Data...")
plot_predictions_with_history(
    original_df=df_hourly, 
    prediction_df=predicted_hourly_prices, 
    interval='hourly', 
    lookback=24 # Showing last 24 hours of history
)

In [ ]:
model_chronos_bolt_base = "/home/laxmidhar/Coding Stuff/AI Models/autogluon-chronos-bolt-base"
model_chronos_bolt_small = "/home/laxmidhar/Coding Stuff/AI Models/autogluon-chronos-bolt-small"

model_amazon_chronos_2 = "/home/laxmidhar/Coding Stuff/AI Models/amazon-chronos-2"

In [ ]:
import pandas as pd
from chronos import Chronos2Pipeline, BaseChronosPipeline

In [ ]:
pipeline: Chronos2Pipeline = BaseChronosPipeline.from_pretrained(model_amazon_chronos_2, device_map="cpu")
df_daily = get_whole_stock_data('J.Kumar Infraprojects Limited', interval="1d")

In [ ]:
df_daily.head()

In [ ]:
feature_df = df_daily.drop(columns="adj_close")
feature_df.head()

In [ ]:
# Generate predictions with covariates
pred_df = pipeline.predict_df(
    df_daily,
    future_df=feature_df,
    prediction_length=10,               # Number of steps to forecast
    quantile_levels=[0.1, 0.5, 0.9],    # Quantiles for probabilistic forecast
    id_column="id",                     # Column identifying different time series
    timestamp_column="datetime",        # Column with datetime information
    target="adj_close",                 # Column(s) with time series values to predict
)
